# N.B. This example is highly out of date please proceed with caution

# Model exporting
Sometimes your model runtime environment may be different your training environment, and this can sometimes involve changing tools, e.g. CMS has a Tensorflow interface built in to its main software package. If your runtime does not yet support PyTorch, you may need to export your trained models in a format that they can then be applied in production.
LUMIN currently has limited, experimental support for exporting to [ONNX](https://github.com/onnx/onnx) (open standard format for representing machine learning models), and [Tensorflow](https://www.tensorflow.org/) Protocolbuffer.

In [1]:
import warnings
import pickle

In [2]:
from pathlib import Path
SAVE_PATH = Path('weights/')

We'll begin by loading a model that was trained during the Binary Classification example

In [3]:
from lumin.nn.models.model import Model

In [4]:
with open(SAVE_PATH/'Binary_Classification_builder.pkl', 'rb') as fin: model_builder = pickle.load(fin)

In [5]:
model = Model.from_save(SAVE_PATH/'Binary_Classification_0.h5', model_builder)

In [6]:
model

Inputs:
30 Continuous: ['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_met_sumet', 'PRI_jet_all_pt', 'PRI_met_px', 'PRI_met_py', 'PRI_jet_leading_px', 'PRI_jet_leading_py', 'PRI_jet_leading_pz', 'PRI_lep_px', 'PRI_lep_py', 'PRI_lep_pz', 'PRI_jet_subleading_px', 'PRI_jet_subleading_py', 'PRI_jet_subleading_pz', 'PRI_tau_px', 'PRI_tau_py', 'PRI_tau_pz', 'PRI_met_pt']
                   
1  Categorical: ['PRI_jet_num']
                   
0  Matrix elements: []
                   

Model:
<bound method Module.parameters of Sequential(
  (0): CatEmbHead(
    (embeds): ModuleList(
      (0): Embedding(4, 2)
    )
  )
  (1): FullyConnected(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): Swish

# ONNX

In order to export to ONNX, we need to hardcode a batch size for the data that will be fed through the model during runtime. Since in a physics analysis, data is normally processed serially, we'll set the batchsize to one. Note: Ensemble also has a `/export2onnx` method which will export all models.

In [7]:
model.export2onnx(str(SAVE_PATH/'Binary_Classification'), bs=1)

OnnxExporterError: Module onnx is not installed!

Now we can load the exported model to check

In [8]:
import onnx
onnx_model = onnx.load(SAVE_PATH/'Binary_Classification.onnx')
onnx.checker.check_model(onnx_model)

In [9]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch-jit-export (
  %0[FLOAT, 1x31]
) initializers (
  %1[FLOAT, 4x2]
  %2[FLOAT, 100x32]
  %3[FLOAT, 100]
  %4[FLOAT, 100x100]
  %5[FLOAT, 100]
  %6[FLOAT, 100x100]
  %7[FLOAT, 100]
  %8[FLOAT, 100x100]
  %9[FLOAT, 100]
  %10[FLOAT, 1x100]
  %11[FLOAT, 1]
) {
  %12 = Slice[axes = [0], ends = [9223372036854775807], starts = [0]](%0)
  %13 = Slice[axes = [1], ends = [9223372036854775807], starts = [30]](%12)
  %14 = Cast[to = 7](%13)
  %15 = Slice[axes = [0], ends = [9223372036854775807], starts = [0]](%14)
  %16 = Constant[value = <Scalar Tensor []>]()
  %17 = Gather[axis = 1](%15, %16)
  %18 = Gather(%1, %17)
  %19 = Concat[axis = 1](%18)
  %20 = Slice[axes = [0], ends = [9223372036854775807], starts = [0]](%0)
  %21 = Slice[axes = [1], ends = [30], starts = [0]](%20)
  %22 = Concat[axis = 1](%21, %19)
  %23 = Gemm[alpha = 1, beta = 1, transB = 1](%22, %2, %3)
  %24 = Sigmoid(%23)
  %25 = Mul(%23, %24)
  %26 = Gemm[alpha = 1, beta = 1, transB = 1](%25, %4, %5)
  %27 = Sigmoid(%

And visualise it with netron to make sure it looks as expected

In [10]:
import netron
netron.start(str(SAVE_PATH/'Binary_Classification.onnx'))

Serving 'weights/Binary_Classification.onnx' at http://localhost:8080


# Tensorflow
Exporting to Tensorflow requires fir exporting to ONNX, then converting the ONNX file to a protocol buffer. Both exports can be performed using the `.export2tfpb` methods.

In [11]:
model.export2tfpb(str(SAVE_PATH/'Binary_Classification.pb'))

/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/__init__.py:87: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use keras.layers.flatten instead.


/home/giles/cernbox/lumin/lumin/nn/models/model.py:423: UserWarning: Tensorflow ProtocolBuffer export of LUMIN models (via ONNX) has not been fully explored or sufficiently tested yet.
                         Please use with caution, and report any trouble
  Please use with caution, and report any trouble""")
/home/giles/cernbox/lumin/lumin/nn/models/model.py:405: UserWarning: ONNX export of LUMIN models has not been fully explored or sufficiently tested yet.
                         Please use with caution, and report any trouble
  Please use with caution, and report any trouble""")
/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/giles/anaconda3

Manually, this involves running:

In [12]:
from onnx_tf.backend import prepare

In [13]:
tf_rep = prepare(onnx_model)

/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of IsInf in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of Mod in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/home/giles/anaconda3/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ThresholdedRelu in d

In [14]:
print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model

['0']
-----
['36']
-----
{'1': <tf.Tensor 'Const:0' shape=(4, 2) dtype=float32>, '2': <tf.Tensor 'Const_1:0' shape=(100, 32) dtype=float32>, '3': <tf.Tensor 'Const_2:0' shape=(100,) dtype=float32>, '4': <tf.Tensor 'Const_3:0' shape=(100, 100) dtype=float32>, '5': <tf.Tensor 'Const_4:0' shape=(100,) dtype=float32>, '6': <tf.Tensor 'Const_5:0' shape=(100, 100) dtype=float32>, '7': <tf.Tensor 'Const_6:0' shape=(100,) dtype=float32>, '8': <tf.Tensor 'Const_7:0' shape=(100, 100) dtype=float32>, '9': <tf.Tensor 'Const_8:0' shape=(100,) dtype=float32>, '10': <tf.Tensor 'Const_9:0' shape=(1, 100) dtype=float32>, '11': <tf.Tensor 'Const_10:0' shape=(1,) dtype=float32>, '0': <tf.Tensor '0:0' shape=(1, 31) dtype=float32>, '12': <tf.Tensor 'Slice:0' shape=(1, 31) dtype=float32>, '13': <tf.Tensor 'Slice_1:0' shape=(1, 1) dtype=float32>, '14': <tf.Tensor 'Cast:0' shape=(1, 1) dtype=int64>, '15': <tf.Tensor 'Slice_2:0' shape=(1, 1) dtype=int64>, '16': <tf.Tensor 'Const_17:0' shape=() dtype=int64>, '1

In [15]:
tf_rep.export_graph(SAVE_PATH/'Binary_Classification.pb')